In [ ]:
pip install torch_geometric Bio

In [ ]:
import re
import os
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt

import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
import torch.optim as optim

from Bio import SeqIO
from io import StringIO
from itertools import product
from tqdm import trange, tqdm
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

from google.colab import drive
drive.mount('/content/drive')

# Define sequence and identifier
sequence = "ATCGATCG"
identifier = "Sequence1"

# Create a Seq object from the sequence
seq_object = Seq(sequence)

# Create a SeqRecord object
seq_record = SeqRecord(seq_object, id=identifier)

print(seq_record)


In [ ]:
def get_kmer_dictionary(kmer_size):
    alphabet = ['A', 'C', 'G', 'T', 'N']
    kmers = [''.join(mer) for mer in product(alphabet, repeat=kmer_size)]
    kmer_dict = {kmer: i for i, kmer in enumerate(kmers)}
    return kmer_dict

def get_edge_index_for_sequence_record(seq_record, kmer_size):
  local_dict = get_kmer_dictionary(kmer_size)
  # print(local_dict)
  sources = []
  targets = []
  for i in range(0, (len(str(seq_record.seq)) - kmer_size - 1)):
    current_block = seq_record.seq[i:i+kmer_size]
    target_block = seq_record.seq[i+1:i+kmer_size+1]
    current_node = local_dict[current_block]
    target_block = local_dict[target_block]

    sources.append(current_node)
    targets.append(target_block)

  return torch.tensor([sources, targets])

  def get_edge_index_for_sequence(seq, kmer_size):
    print("here")
    local_dict = get_kmer_dictionary(kmer_size)
    # print(local_dict)
    sources = []
    targets = []
    for i in range(0, (len(seq) - kmer_size - 1)):
      current_block = seq[i:i+kmer_size]
      target_block = seq[i+1:i+kmer_size+1]
      current_node = local_dict[current_block]
      target_block = local_dict[target_block]

      sources.append(current_node)
      targets.append(target_block)

    return torch.tensor([sources, targets])

In [ ]:
%cd /content/drive/MyDrive/10708/10708 Project

In [ ]:
#FRAGMENTED SEQUENCES NOT FULL

from tqdm import tqdm
fasta_file = "sequences.fasta"

sequences = []
records = list(SeqIO.parse("sequences.fasta", "fasta"))


def get_edge_index_for_sequence(seq, kmer_size):
    local_dict = get_kmer_dictionary(kmer_size)
    # print(local_dict)
    sources = []
    targets = []
    for i in range(0, (len(seq) - kmer_size - 1)):
      current_block = seq[i:i+kmer_size]
      target_block = seq[i+1:i+kmer_size+1]
      current_node = local_dict[current_block]
      target_block = local_dict[target_block]

      sources.append(current_node)
      targets.append(target_block)

    return torch.tensor([sources, targets])
def get_subsequences(sequence, subseq_length):
    """Generate subsequences of the given length from a sequence."""
    results = []
    for i in range(len(sequence) - subseq_length + 1):
        if i + subseq_length > len(sequence):
          break
        results.append(sequence[i:i + subseq_length])
    return results

train_data = []
subseq_length = 100 # Length of each subsequence
kmer_size = 3      # Size of k-mer

for record in records:
    # Retrieve the sequence from the record
    sequence = str(record.seq)
    # For each subsequence of length 50:
    subseqs = get_subsequences(sequence, subseq_length)
    for i in tqdm(range(len(subseqs))):
        subseq = subseqs[i]
        edge_index = get_edge_index_for_sequence(subseq, kmer_size)
        train_data.append(edge_index)


In [ ]:
fasta_file = "sequences.fasta"

sequences = []
records = list(SeqIO.parse("sequences.fasta", "fasta"))

print(get_edge_index_for_sequence(records[0], 1).shape)

train_data = []
for record in records:
  train_data.append(get_edge_index_for_sequence(record, 2))

trunc_size = min(dat.shape[1] for dat in train_data)
# print(trunc_size)

#Truncate all the sequences to trunc_size - 5
for i in range(len(train_data)):
  train_data[i] = train_data[i][:,:trunc_size-5]

for dat in train_data:
  print(dat.shape)

In [ ]:
data_tensor = torch.stack(train_data, dim=0)


In [ ]:
print(data_tensor.shape)

In [ ]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, out_channels, num_edges, num_nodes):
        super(GCN, self).__init__()

        self.num_nodes = num_nodes

        # Initialize GCNConv layers
        self.conv1 = GCNConv(in_channels, 1)

        self.conv_mu = GCNConv(1, out_channels)
        self.conv_logvar = GCNConv(1, out_channels)

        # Decoder for adjacency matrix
        self.decoder_linear = torch.nn.Linear(num_nodes, 16)
        self.decoder_ReLU = torch.nn.ReLU()
        self.decoder_linear2 =  torch.nn.Linear(16, num_edges)

    def reparameterize(self, mu, logvar):
        std = torch.exp(logvar / 2)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        mu = self.conv_mu(x, edge_index)
        logvar = self.conv_logvar(x, edge_index)
        z = self.reparameterize(mu, logvar).T

        # print("z shape after reparameterizing", z.shape)
        decoder1 = self.decoder_linear(z)
        # print("z shape after linear", z.shape)
        decoder2 = self.decoder_ReLU(decoder1)
        # print("z shape are relu", z.shape)
        decoder_edge_index = self.decoder_linear2(decoder2)
        # print("decoder_edge_index shape", decoder_edge_index.shape)
        return decoder_edge_index, mu, logvar, z

    def encode(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        mu = self.conv_mu(x, edge_index)
        logvar = self.conv_logvar(x, edge_index)
        z = self.reparameterize(mu, logvar).T
        return mu, logvar, z

    def decode(self, z):
        decoder1 = self.decoder_linear(z)
        decoder2 = self.decoder_ReLU(decoder1)
        decoder_edge_index = self.decoder_linear2(decoder2)
        return decoder_edge_index


def loss_function(mse_loss_fn, recon_edge_index, true_edge_index, mu, logvar):
   # Initialize here
    BCE_adj = mse_loss_fn(recon_edge_index, true_edge_index)  # Correctly apply the loss function
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE_adj + KLD



In [ ]:
#global data
num_nodes = len(get_kmer_dictionary(2))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN(1, 2, 29780, num_nodes)
model.load_state_dict(torch.load("/content/drive/MyDrive/10708/10708 Project/VAE.pt"))


In [ ]:
train_embeddings = []
for edge_index in train_data:
  x = torch.randn(num_nodes, 1)
  _,_,z = model.encode(x, edge_index)
  train_embeddings.append(z.flatten())
  print(z.flatten().shape)

data_tensor = torch.stack(train_embeddings)

In [ ]:
import torch
import torch.nn as nn

import torch
import torch.nn as nn

class Discriminator(nn.Module):
    def __init__(self, embedding_dim):
        super(Discriminator, self).__init__()
        # Define each layer separately

        self.linear1 = nn.Linear(embedding_dim, 256)
        self.leaky_relu1 = nn.LeakyReLU(0.2)
        self.linear2 = nn.Linear(256, 128)
        self.leaky_relu2 = nn.LeakyReLU(0.2)
        self.linear3 = nn.Linear(128, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):

        # Manually connect each layer
        # print("discriminator a", x.shape)
        x = self.leaky_relu1(self.linear1(x))
        # print("discriminator b", x.shape)
        x = self.leaky_relu2(self.linear2(x))
        x = self.sigmoid(self.linear3(x))

        return x

# Example usage:
embedding_dim = 50  # Example embedding dimension
discriminator = Discriminator(embedding_dim)

# Example input tensor (batch size of 1)
x = torch.randn(1, embedding_dim)

# Generate output
output = discriminator(x)
print("Output:", output)


In [ ]:
import torch
import torch.nn as nn

class Generator(nn.Module):
    def __init__(self, input_dim, embedding_dim):
        super(Generator, self).__init__()
        self.embedding_dim = embedding_dim

        # Define each layer separately
        self.linear1 = nn.Linear(input_dim, 128)
        self.leaky_relu1 = nn.LeakyReLU(0.2)
        self.linear2 = nn.Linear(128, 256)
        self.leaky_relu2 = nn.LeakyReLU(0.2)
        self.linear3 = nn.Linear(256, embedding_dim)
        self.tanh = nn.Softplus()

    def forward(self, z):
        # Manually connect each layer
        # print("a", z.shape)
        z = self.leaky_relu1(self.linear1(z))
        # print("b", z.shape)
        z = self.leaky_relu2(self.linear2(z))
        z = self.tanh(self.linear3(z))

        # Reshape the output to match the desired dimensions
        return z

# Example usage:
input_dim = 100  # Size of the input latent vector
embedding_dim = 50  # Ensure this is divisible by 2 as it needs to be reshaped into two columns
generator = Generator(input_dim, embedding_dim)

# Generate a random noise vector
z = torch.randn(1, input_dim)

# Generate output
output = generator(z)
print("Output shape:", output.shape)

# Example usage:
latent_dim = 100
embedding_dim = 50  # This is the 25 in your data shape [25, 2]
generator = Generator(input_dim=latent_dim, embedding_dim=50)


In [ ]:
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))
g_optimizer = torch.optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))


In [ ]:
num_samples = data_tensor.size(0)
split_idx = int(num_samples * 0.8)

# Split the tensor
train_tensor = data_tensor[:split_idx]
test_tensor = data_tensor[split_idx:]

In [ ]:
print(train_tensor.shape)

In [ ]:
num_epochs = 5000
for epoch in range(num_epochs):
    for real_data in train_tensor:  # Assuming real_data is loaded from somewhere
        batch_size = 1
        real_labels = torch.ones(batch_size, 1)
        fake_labels = torch.zeros(batch_size, 1)  # Correct the size to match real_labels

        # Train Discriminator with real data
        d_optimizer.zero_grad()
        real_data = real_data.unsqueeze(0) if len(real_data.shape) == 1 else real_data  # Ensure real_data is at least 2D
        outputs = discriminator(real_data)
        d_loss_real = criterion(outputs, real_labels)
        # d_loss_real.backward(retain_graph=True)  # retain_graph as further backward calls are expected

        # Train Discriminator with fake data
        noise = torch.randn(batch_size, latent_dim)
        fake_data = generator(noise)
        outputs = discriminator(fake_data.detach())  # detach to avoid training generator now
        d_loss_fake = criterion(outputs, fake_labels)
        d_loss_fake.backward(retain_graph=True)
        d_optimizer.step()

        # Train Generator
        g_optimizer.zero_grad()
        outputs = discriminator(fake_data)  # no detach here, we want gradients to flow here
        g_loss = criterion(outputs, real_labels)  # Trick the discriminator
        g_loss.backward()  # no need to retain_graph here as it's the last backward call in the loop
        g_optimizer.step()

    print(f"Epoch {epoch+1}/{num_epochs}, D Loss: {d_loss_real.item() + d_loss_fake.item()}, G Loss: {g_loss.item()}")


In [ ]:
model = GCN(1, 2, 29780, num_nodes)
model.load_state_dict(torch.load("/content/drive/MyDrive/10708/10708 Project/VAE.pt"))

generated_sequences = []
kmer_dict = get_kmer_dictionary(2)
int_to_kmer = {value: key for key, value in kmer_dict.items()}


for i in range(1):
  noise = torch.randn(1, 100)
  fake_data = generator(noise)
  outputs = discriminator(fake_data.detach())

  generated_edge_list = (model.decode(fake_data.reshape(2,25)))

  generated_seq = ""
  for i in range(25): #this should be a straight path, i.e. i can just read the first row
    node = int(generated_edge_list[0][i]) #going to have to round
    kmer = int_to_kmer[node]
    generated_seq += (kmer)
  generated_sequences.append(generated_seq)


In [ ]:
generated_sequences